<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix</h1>

In [ ]:
!pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

# Launch Phoenix

In [ ]:
import phoenix as px

session = px.launch_app()

# Set Up OTel

In [ ]:
from phoenix.otel import register

tracer_provider = register(endpoint="http://localhost:6006/v1/traces")

# Instrument CrewAI

In [ ]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

CrewAI uses either Langchain or LiteLLM under the hood to call LLMs, depending on the package version. We recommend instrumenting the corresponding library to get visibility into the LLM calls.

In [ ]:
! pip show crewai | grep Version

If you're using CrewAI<0.63.0, instrument Langchain:

In [ ]:
# ! pip install openinference-instrumentation-langchain

# from openinference.instrumentation.langchain import LangChainInstrumentor

# LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

If you're using CrewAI>=0.63.0, instrument LiteLLM:

In [ ]:
! pip install openinference-instrumentation-litellm

from openinference.instrumentation.litellm import LiteLLMInstrumentor

LiteLLMInstrumentor().instrument(tracer_provider=tracer_provider)

# Keys

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)






In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    # You can pass an optional llm attribute specifying what model you wanna use.
    # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
    tools=[search_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer], tasks=[task1, task2], verbose=1, process=Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

# View Results

This guide is packed in a neat notebook for convenience, however many people will host Arize Phoenix in a [Docker](https://docs.arize.com/phoenix/deployment/docker) Container or Spin up an instance on their local machine.

In [ ]:
print(f"View traces at {session.url}")

 # ⭐⭐⭐ If you like this guide, please give [Arize Phoenix](https://github.com/Arize-ai/phoenix) a star ⭐⭐⭐